<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Deployment of a model to Azure App Service and setting CORS policies

## Table of contents

1. [Introduction](#intro)
1. [Model retrieval](#model)
1. [Scoring script](#scoring)
1. [Environment retrieval](#environment)
1. [Image Build](#image_build)
1. [Model deployment as a web application](#deploy)
  1. [App service plan](#app_service_plan)
  1. [Web application](#web_app)
  1. [ACR credentials](#acr_cred)
  1. [ACR authentication](#acr_auth)
1. [Monitor deployment](#monitor)
  1. [Azure Portal](#azure_portal)
  1. [Azure CLI](#azure_cli)
1. [Test deployment](#test)
1. [CORS](#cors)
1. [Clean up](#clean_up)

## 1. Introduction <a id="intro"/>

Azure App Service allows users to build and host web apps, mobile back ends, and RESTful APIs in the programming language of one's choice without managing infrastructure. It offers auto-scaling and high availability with support for both Windows and Linux. It also allows for automated deployments from Github, Azure DevOps, or any Git repo. 

For our purposes, we will be using its support for hosting RESTful APIs to deploy our trained machine learning model to a RESTful API endpoint.

In this tutorial, we will learn how to target Azure Apps as a deployment target using the trained model and resources from our previous notebook [21_deployment_on_azure_container_instances.ipynb](21_deployment_on_azure_container_instances.ipynb). 

We will also learn how to set Cross-Origin Resource Sharing (CORS) policies that is required when developing a web application hosted in a different domain from the one hosting the deployed machine learning model's RESTful APIs.

To learn more about Cross-Origin Resource Sharing (CORS) policies, go to the [CORS section](#CORS) of this notebook.

### Pre-requisites <a id="pre-reqs"/>

This notebook relies on resources we created in [21_deployment_on_azure_container_instances.ipynb](21_deployment_on_azure_container_instances.ipynb):
- Our Azure Machine Learning workspace
- Our Azure Machine Learning environment that we registered to the workspace which contains the conda dependencies and the requirements for building the docker image that we used for deploying the model to Azure Container Instances.
- The model and scoring script needed for the web service to work.




If you are missing any of these, you should go back and run the steps from the sections "Pre-requisites" to "3.D Environment setup" to generate them.



### Library import

Let's first import the libraries that we will need for this notebook.

In [1]:
# Let's first import required modules for this notebook.
import json
import requests
import sys
# This "sys.path.extend()" statement allows us to move up the directory hierarchy 
# and access the Computer Vision Repository utils_cv package
sys.path.extend([".", "../.."])

# Import AzureML modules that are required for this notebook
from azureml.core import Workspace, Environment
from azureml.core.model import Model, InferenceConfig

# Import custom vision utilities for retrieving workspaces and for testing
from utils_cv.common.azureml import get_or_create_workspace
from utils_cv.common.data import data_path
from utils_cv.common.image import ims2strlist

For us to deploy our model to Azure App Services, we first need to retrieve the Azure Machine Learning workspace that we created in previous notebooks (or create one if we have not created one yet). 

You will need the following information to create or retrieve an AzureML workspace:
- subscription_id: The ID of the Azure subscription you wish to use for the workspace.
- resource_group: The name of the resource group to use for the workspace.
- workspace_name: The name you wish to use for this workspace.
- workspace_region: The geographical area to use for this workspace (eastus, eastus2, etc).

In [2]:
subscription_id = "YOUR_SUBSCRIPTION_ID"
resource_group = "YOUR_RESOURCE_GROUP_NAME"
workspace_name = "YOUR_WORKSPACE_NAME"
workspace_region = "YOUR_WORKSPACE_REGION" #Possible values eastus, eastus2 and so on.

If you have not created an Azure Machine Learning workspace yet and have specific questions around workspaces, please refer to [AzureML's official documentation page](https://docs.microsoft.com/en-us/python/api/overview/azure/ml/?view=azure-ml-py#workspace)

An AzureML workspace contains the various resources that are required across Azure Machine Learning Services, from training to inferencing and is required for this notebook.

In [3]:
ws = get_or_create_workspace(
                subscription_id,
                resource_group,
                workspace_name,
                workspace_region)

# Let's print the workspace details
print("Workspace name: " + ws.name,
      "Workspace region: " + ws.location,
      "Subscription ID: " + ws.subscription_id,
      "Resource Group: " + ws.resource_group, sep = "\n")

Workspace name: cvws
Workspace region: eastus
Subscription ID: 0ca618d2-22a8-413a-96d0-0f1b531129c3
Resource Group: cvbp_project_resources


## 2. Retrieve registered model<a id="model" />

We will now retrieve the model that we registered to the workspace using the name that we gave `im_classif_resnet18` in [21_deployment_on_azure_container_instances.ipynb](21_deployment_on_azure_container_instances.ipynb)

In [4]:
model = Model(ws, "im_classif_resnet18")

## 3. Scoring script <a id="scoring" />

We will reuse the scoring script that we created in [21_deployment_on_azure_container_instances.ipynb](21_deployment_on_azure_container_instances.ipynb).

If you have created a scoring script from the linked notebook, you can skip to the next section: [Environment retrieval](#environment)

### (optional, only required if you have not created one yet)

For the web service to return predictions on a given input image, we need to provide it with instructions on how to use the model we just registered. These instructions are stored in the scoring script.

This script must contain two required functions, init() and run(input_data):

- In the init() function, we typically load the model into a global object. This function is executed only once when the Docker container is started.
- In the run(input_data) function, the model is used to predict a value based on the input data. The input and output of run typically use JSON as serialization and de-serialization format but we are not limited to that.

This file must also be stored in the current directory.

In [5]:
scoring_script = "score.py"

In [6]:
%%writefile $scoring_script
# Copyright (c) Microsoft. All rights reserved.
# Licensed under the MIT license.

import os
import json

from base64 import b64decode
from io import BytesIO

from azureml.core.model import Model
from fastai.vision import load_learner, open_image

def init():
    global model
    model_path = Model.get_model_path(model_name='im_classif_resnet18')
    # ! We cannot use the *model_name* variable here otherwise the execution on Azure will fail !

    model_dir_path, model_filename = os.path.split(model_path)
    model = load_learner(model_dir_path, model_filename)


def run(raw_data):

    # Expects raw_data to be a list within a json file
    result = []    
    
    for im_string in json.loads(raw_data)['data']:
        im_bytes = b64decode(im_string)
        try:
            im = open_image(BytesIO(im_bytes))
            pred_class, pred_idx, outputs = model.predict(im)
            result.append({"label": str(pred_class), "probability": str(outputs[pred_idx].item())})
        except Exception as e:
            result.append({"label": str(e), "probability": ''})
    return result

Overwriting score.py


## 4. Retrieve registered environment<a id="environment" />

We will retrieve the environment that we registered to the workspace for reuse in our deployment to Azure App Services.

The retrieved environment will contain the conda packages and dependencies that we specified in [21_deployment_on_azure_container_instances.ipynb](21_deployment_on_azure_container_instances.ipynb). 

It will also contain the docker specifications such as the base dockerfile to use to build the image and the required inferencing packages that are needed in the resulting docker image for the deployment to work.

NOTE: As mentioned before, we need to create a conda environment that contains the required dependencies for the trained model. Normally, this means that we need to create a conda environment that closely matches the one in which the model was trained.

For additional details on using environments, please refer to [AzureML's documentation page on environments](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments).

In [7]:
inferencing_env = Environment.get(workspace=ws, name="im_classif_resnet18")

## 5. Build the image used for deployment<a id="image_build" />

Let's now create an `InferenceConfig` using the environment we just retrieved and the scoring script.

We will then package up the model, the scoring script and the environment (containing the conda dependencies that we specified) to build the final image that will be used for deployment.

The built image will be pushed into the Azure Container Registry (ACR) for the workspace, which we will be using in later steps.

We can monitor the image build process through the streamed logs.

NOTE: It may take a minute or two for the docker image build logs to start streaming.

In [8]:
inference_config = InferenceConfig(entry_script="score.py", environment=inferencing_env)

inference_image = Model.package(ws, [model], inference_config)
# Setting show_output to True to stream the logs from the Docker image build process
inference_image.wait_for_creation(show_output=True)

2020/04/10 11:33:12 Downloading source code...
2020/04/10 11:33:14 Finished downloading source code
2020/04/10 11:33:14 Creating Docker network: acb_default_network, driver: 'bridge'
2020/04/10 11:33:15 Successfully set up Docker network: acb_default_network
2020/04/10 11:33:15 Setting up Docker configuration...
2020/04/10 11:33:16 Successfully set up Docker configuration
2020/04/10 11:33:16 Logging in to registry: cvwsbbd59dea.azurecr.io
2020/04/10 11:33:17 Successfully logged into cvwsbbd59dea.azurecr.io
2020/04/10 11:33:17 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2020/04/10 11:33:17 Launching container with name: acb_step_0
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /acb/home/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login S

 ---> Running in 0fece4105885
Removing intermediate container 0fece4105885
 ---> 77a7a365e29f
Step 7/8 : RUN mv '/var/azureml-app/tmpsj7sj0t4.py' /var/azureml-app/main.py
 ---> Running in 9689a888ad8a
Removing intermediate container 9689a888ad8a
 ---> b4f18a88d2b9
Step 8/8 : CMD ["runsvdir","/var/runit"]
 ---> Running in 52040ed3810c
Removing intermediate container 52040ed3810c
 ---> b8cf162127fc
Successfully built b8cf162127fc
Successfully tagged cvwsbbd59dea.azurecr.io/azureml/azureml_84830b7adc0a892fa927d004f3b1fcb2:latest
2020/04/10 11:35:29 Successfully executed container: acb_step_1
2020/04/10 11:35:29 Executing step ID: acb_step_2. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2020/04/10 11:35:29 Pushing image: cvwsbbd59dea.azurecr.io/azureml/azureml_84830b7adc0a892fa927d004f3b1fcb2:latest, attempt 1
The push refers to repository [cvwsbbd59dea.azurecr.io/azureml/azureml_84830b7adc0a892fa927d004f3b1fcb2]
2179889df0f6: Preparing
b45f748a732a: Preparing


Now let's print the location of the built image on Azure Container Registry (ACR) so that we can use it to deploy our model to Azure App Services.

In [9]:
image_location = inference_image.location
print("image location on ACR: {}".format(image_location))

image location on ACR: cvwsbbd59dea.azurecr.io/azureml/azureml_84830b7adc0a892fa927d004f3b1fcb2@sha256:4e3b949aea1fd9177ea5f53ecf1a766c9ba5c61e912c4c00b17eb9376ac26708


## 6. Deploy image as a web application<a id="deploy" />

These steps require the Azure CLI, so please make sure you have installed it. 

If you haven't, please install it following the [Azure CLI installation documentation](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli?view=azure-cli-latest).

Let's run `az login` to make sure that the Azure CLI is installed and that you're logged into Azure.

In [10]:
!az login

### 6.A. Create an app service plan to deploy the service<a id="app_service_plan" />

Now that we've retrieved the credentials for the ACR that hosts the image we'll be using to deploy to Azure App Services, we'll now create an app service plan for the deployment.

We'll use the P1v2 (`--sku P1v2`) SKU and since images created by Azure Machine Learning Services are linux-based, we need to specify (`--is-linux`) to indicate that we'll be using a linux based image.

The full list of other SKUs that are available along with their pricing can be found at [https://azure.microsoft.com/en-us/pricing/details/app-service/linux/](https://azure.microsoft.com/en-us/pricing/details/app-service/linux/)

In [11]:
appservice_plan_name = "im_classif_webapp_plan"

You will see the JSON output of the created app service plan once the creation is successful.

In [12]:
!az appservice plan create --resource-group {resource_group} --name {appservice_plan_name} --sku P1v2 --is-linux

{
  "freeOfferExpirationTime": null,
  "geoRegion": "Central US",
  "hostingEnvironmentProfile": null,
  "hyperV": false,
  "id": "/subscriptions/0ca618d2-22a8-413a-96d0-0f1b531129c3/resourceGroups/cvbp_project_resources/providers/Microsoft.Web/serverfarms/im_classif_webapp_plan",
  "isSpot": false,
  "isXenon": false,
  "kind": "linux",
  "location": "Central US",
  "maximumElasticWorkerCount": 1,
  "maximumNumberOfWorkers": 30,
  "name": "im_classif_webapp_plan",
  "numberOfSites": 0,
  "perSiteScaling": false,
  "provisioningState": "Succeeded",
  "reserved": true,
  "resourceGroup": "cvbp_project_resources",
  "sku": {
    "capabilities": null,
    "capacity": 1,
    "family": "Pv2",
    "locations": null,
    "name": "P1v2",
    "size": "P1v2",
    "skuCapacity": null,
    "tier": "PremiumV2"
  },
  "spotExpirationTime": null,
  "status": "Ready",
  "subscription": "0ca618d2-22a8-413a-96d0-0f1b531129c3",
  "tags": null,
  "targetWorkerCount": 0,
  "targetWorkerSizeId": 0,
  "type"

### 6.B. Create and deploy a web app that hosts REST APIs using our trained machine learning model.<a id="web_app" />

We will now create and deploy a web app that will host the RESTful APIs for our trained machine learning model using the docker image that we built and pushed to ACR.

NOTE: The name used for the web app must be able to produce a unique Fully Qualified Domain Name (FQDN) as AppName.azurewebsites.net.


NOTE: The name of your image on your workspace ACR has been retrieved for your convenience, but you can find it yourself by looking at the printed out image location which is in the format: `<acr_name>.azurecr.io/azureml/azureml_a1234567@sha256:<image_name>`. 

The name of the image is the string that you'll find in `<image_name>`

In [13]:
webapp_name = "im-classif-resnet18-webapp"

You will see the JSON output of the created web app once the creation is successful.

In [14]:
!az webapp create --resource-group {resource_group} --plan {appservice_plan_name} --name {webapp_name} --deployment-container-image-name {image_location}

{
  "availabilityState": "Normal",
  "clientAffinityEnabled": true,
  "clientCertEnabled": false,
  "clientCertExclusionPaths": null,
  "cloningInfo": null,
  "containerSize": 0,
  "dailyMemoryTimeQuota": 0,
  "defaultHostName": "im-classif-resnet18-webapp.azurewebsites.net",
  "enabled": true,
  "enabledHostNames": [
    "im-classif-resnet18-webapp.azurewebsites.net",
    "im-classif-resnet18-webapp.scm.azurewebsites.net"
  ],
  "ftpPublishingUrl": "ftp://waws-prod-dm1-171.ftp.azurewebsites.windows.net/site/wwwroot",
  "geoDistributions": null,
  "hostNameSslStates": [
    {
      "hostType": "Standard",
      "ipBasedSslResult": null,
      "ipBasedSslState": "NotConfigured",
      "name": "im-classif-resnet18-webapp.azurewebsites.net",
      "sslState": "Disabled",
      "thumbprint": null,
      "toUpdate": null,
      "toUpdateIpBasedSsl": null,
      "virtualIp": null
    },
    {
      "hostType": "Repository",
      "ipBasedSslResult": null,
      "ipBasedSslState": "NotConfigu

With the above command, your webapp has been created! You will be able to inspect the printed JSON output to learn more details about the created webapp.

However, since you haven't provided the credentials to the Azure Container Registry where the docker image that you'll be using is hosted, the web app is not functional as it does not have access to the ACR and the image in the registry.

### 6.C. Retrieve credentials for your Azure Container Registry (ACR) registered to the AzureML workspace<a id="acr_cred" />

Now that we are logged into the Azure CLI and we know the name of the ACR that our deployment image is uploaded to, let's retrieve the credentials for the ACR using Azure CLI commands.

NOTE: The name of your workspace ACR has been retrieved for your convenience, but you can find it yourself by looking at the printed out image location which is in the format: `<acr_name>.azurecr.io/azureml/azureml_a1234567@sha256:<image_name>`. 

The name of the ACR is the string that you'll find in `<acr_name>`

In [15]:
acr_name = image_location.split(".")[0]
print("acr_name: {}".format(acr_name))

acr_name: cvwsbbd59dea


In [16]:
!az acr credential show --name {acr_name}

In [17]:
acr_username = "YOUR-ACR-USERNAME"
acr_password = "YOUR-ACR-PASSWORD"

### 6.D. Webapp authentication for the workspace ACR<a id="acr_auth" />

We will now provide the credentials that we retrieved to the web app to authenticate it for the workspace ACR.

This will allow the web app to pull and load the docker image from the workspace ACR to host the scoring REST API.

In [18]:
acr_server_url = image_location.split("/")[0]
print("acr_server_url: {}".format(acr_server_url))

acr_server_url: cvwsbbd59dea.azurecr.io


You will see the JSON output of the provided container setting once the authentication is successful.

In [19]:
!az webapp config container set --resource-group {resource_group} --name {webapp_name} --docker-custom-image-name {image_location} --docker-registry-server-url {acr_server_url} --docker-registry-server-user {acr_username} --docker-registry-server-password {acr_password}

[
  {
    "name": "WEBSITES_ENABLE_APP_SERVICE_STORAGE",
    "slotSetting": false,
    "value": "false"
  },
  {
    "name": "DOCKER_REGISTRY_SERVER_URL",
    "slotSetting": false,
    "value": "cvwsbbd59dea.azurecr.io"
  },
  {
    "name": "DOCKER_REGISTRY_SERVER_USERNAME",
    "slotSetting": false,
    "value": "cvwsbbd59dea"
  },
  {
    "name": "DOCKER_REGISTRY_SERVER_PASSWORD",
    "slotSetting": false,
    "value": null
  },
  {
    "name": "DOCKER_CUSTOM_IMAGE_NAME",
    "value": "DOCKER|cvwsbbd59dea.azurecr.io/azureml/azureml_84830b7adc0a892fa927d004f3b1fcb2@sha256:4e3b949aea1fd9177ea5f53ecf1a766c9ba5c61e912c4c00b17eb9376ac26708"
  }
]


## 7. Monitor the progress of pulling and loading the deployment image<a id="monitor" />

Now that the web app has access to the ACR hosting the docker image, it will start pulling and loading the docker image onto the web app.

We can monitor the progress by viewing the web app's logs. Let's first turn on container logging to access the console logs generated from inside the container.

In [20]:
!az webapp log config --resource-group {resource_group} --name {webapp_name} --docker-container-logging filesystem

{
  "applicationLogs": {
    "azureBlobStorage": {
      "level": "Off",
      "retentionInDays": null,
      "sasUrl": null
    },
    "azureTableStorage": {
      "level": "Off",
      "sasUrl": null
    },
    "fileSystem": {
      "level": "Off"
    }
  },
  "detailedErrorMessages": {
    "enabled": false
  },
  "failedRequestsTracing": {
    "enabled": false
  },
  "httpLogs": {
    "azureBlobStorage": {
      "enabled": false,
      "retentionInDays": 3,
      "sasUrl": null
    },
    "fileSystem": {
      "enabled": true,
      "retentionInDays": 3,
      "retentionInMb": 100
    }
  },
  "id": "/subscriptions/0ca618d2-22a8-413a-96d0-0f1b531129c3/resourceGroups/cvbp_project_resources/providers/Microsoft.Web/sites/im-classif-resnet18-webapp/config/logs",
  "kind": null,
  "location": "Central US",
  "name": "logs",
  "resourceGroup": "cvbp_project_resources",
  "type": "Microsoft.Web/sites/config"
}


Let's also make sure webserver logging is set to `filesystem` to ensure that we also save the logs from the webserver to the filesystem.

In [21]:
!az webapp log config --resource-group {resource_group} --name {webapp_name} --web-server-logging filesystem

{
  "applicationLogs": {
    "azureBlobStorage": {
      "level": "Off",
      "retentionInDays": null,
      "sasUrl": null
    },
    "azureTableStorage": {
      "level": "Off",
      "sasUrl": null
    },
    "fileSystem": {
      "level": "Off"
    }
  },
  "detailedErrorMessages": {
    "enabled": false
  },
  "failedRequestsTracing": {
    "enabled": false
  },
  "httpLogs": {
    "azureBlobStorage": {
      "enabled": false,
      "retentionInDays": 3,
      "sasUrl": null
    },
    "fileSystem": {
      "enabled": true,
      "retentionInDays": 3,
      "retentionInMb": 100
    }
  },
  "id": "/subscriptions/0ca618d2-22a8-413a-96d0-0f1b531129c3/resourceGroups/cvbp_project_resources/providers/Microsoft.Web/sites/im-classif-resnet18-webapp/config/logs",
  "kind": null,
  "location": "Central US",
  "name": "logs",
  "resourceGroup": "cvbp_project_resources",
  "type": "Microsoft.Web/sites/config"
}


### 7.A. Monitoring using the Azure Portal<a id="azure_portal" />

You can view the status of your webapp from the Azure Portal: [https://portal.azure.com](https://portal.azure.com)

On the Azure Portal, search for the webapp that you just created using its name `im-classif-resnet18-webapp`.

<img src="https://raw.githubusercontent.com/microsoft/computervision-recipes/staging/scenarios/classification/media/deployment/azure_portal.PNG" />

Now, let's try taking a look at the status of the web app from the portal and make sure that the webapp is up and running.

<img src="https://raw.githubusercontent.com/microsoft/computervision-recipes/staging/scenarios/classification/media/deployment/webapp_status.PNG" />

Now that we have made sure that the web app is running, let's take a look at the logs by going to the `Container settings` section.

Set the `Image source` as `Private Registry` and make sure the details of the registry and image match the values specified in the above steps.

Once the image has been loaded and the web app is fully functional, the log will display a message stating:

`Container <image_location> for site <webapp_name> initialized successfully and is ready to serve requests.`

NOTE: Due to the size of the image it may take between 10-15 minutes for the pulling, extracting and loading of the docker image to complete. Also, the logs do not auto refresh, so you will need to manually refresh to pull the latest logs.

<img src="https://raw.githubusercontent.com/microsoft/computervision-recipes/staging/scenarios/classification/media/deployment/webapp_logs.PNG" />

### 7.B. Monitoring using streamed logs via Azure CLI<a id="azure_cli" />

We can also stream the logs from the webapp using the Azure CLI commands. This allows us to more easily view the webapp's ongoing processes without having to manually refresh the logs. This is the easier and preferred way to monitor the deployment process.

Run the command below from a shell program:

`az webapp log tail --resource-group <YOUR-RESOURCE-GROUP> --name <YOUR-WEBAPP-NAME>`

NOTE: Running the above command in a notebook using `!` will not work as running bash shells in jupyter notebooks will not display any output until everything is finished running.

Once you see in the logs the following lines indicating that the deployment was successful, you can stop/kill the process as it will continue to listen for new logs from the deployed service. Stopping the process WILL NOT interrupt the service or cause it to stop, so there's no need to worry as it will only stop listening for additional logs.

`2020-04-10 12:08:02.010 INFO  - Container im-classif-resnet18-webapp_0_ef19ced8 for site im-classif-resnet18-webapp initialized successfully and is ready to serve requests.`

NOTE: If you run into issues in log streaming, fall back to [monitoring using the Azure Portal](#azure_portal) which may provide additional details.

## 8. Test the scoring REST API endpoint for the deployed web app<a id="test" />

We will now test the scoring endpoint for the deployed web app using the test images hosted on Azure Blob Storage and then pre-process them using the utility function `ims2strlist()` into string decoded base64 encoded image bytes.

In [22]:
test_image_directory = "https://cvbp-secondary.z19.web.core.windows.net/images/"
test_image_filenames = ["cvbp_milk_bottle.jpg", "cvbp_water_bottle.jpg"]
local_test_image_paths = []

for test_image_filename in test_image_filenames:
    req = requests.get(os.path.join(test_image_directory, test_image_filename))
    local_test_image_path = os.path.join(data_path(), test_image_filename)

    with open(local_test_image_path, "wb") as file:
        file.write(req.content)
        local_test_image_paths.append(local_test_image_path)

# Use the utility function im2strlist to get a list containing base64-encoded images decoded into strings
decoded_b64_test_images = ims2strlist(local_test_image_paths)

With the two test images that we downloaded in the step above, let's test the scoring endpoint using the `requests` library.

You should get the predictions back in the format of a JSON string, containing the labels and their probabilities for the test images.

If the labels for the two test images came back as `water_bottle`, then congratulations, your scoring endpoint using the pretrained machine learning model is now fully up and running!

In [23]:
scoring_uri = "https://{}.azurewebsites.net/score".format(webapp_name)

headers = {"Content-Type" : "application/json"}

test_image_data = json.dumps({"data" : decoded_b64_test_images})

response = requests.post(scoring_uri, data=test_image_data, headers=headers)

print("Predictions: {}".format(response.json()),
      "Received the scored result in: {}".format(response.elapsed),
      "Response status code: {}".format(response.status_code), sep="\n")

Predictions: [{'label': 'water_bottle', 'probability': '0.800184428691864'}, {'label': 'water_bottle', 'probability': '0.6857761144638062'}]
Received the scored result in: 0:00:01.063833
Response status code: 200


## 9. Cross-Origin Resource Sharing (CORS)<a id="cors"/>

Cross-Origin Resource Sharing (CORS) is an HTTP feature that allows a web application running under a single domain to access resources from another domain. 

By default, web browsers enable [same origin policy](https://www.w3.org/Security/wiki/Same_Origin_Policy), a security mechanism to prevent webpages from calling APIs from a different domain than the one it is calling from. Under the same origin policy, one origin is allowed to send information to another origin, but is not permitted to receive information from another origin to prevent malicious websites from accessing confidential information from other websites.

CORS provides a secure way to allow one origin to both call and receive information from APIs that are hosted in a different origin.

**Hence, setting the correct CORS policy is required if you are building a web application that is hosted in a different domain from the one hosting the deployed machine learning model's RESTful APIs.**

To learn more about CORS in detail, please refer to either the [CORS specification from World Wide Web Consortium (W3C)](https://www.w3.org/TR/cors/) or [Mozilla's documentation page for developers](https://developer.mozilla.org/en-US/docs/Web/HTTP/CORS)

In [24]:
# To allow multiple origins, add them to the allowed origins list. To allow all client URLs, add the wildcard '*'
origins = "\"['YOUR-WEBAPP-ORIGIN']\"" # The string format for allowed origins HAS to be in the format "['abc']" 
print("Allowed origins: {}".format(origins))

Allowed origins: "['YOUR-WEBAPP-ORIGIN']"


Make sure you see the allowedOrigins configuration correctly set in `["properties"]["cors"]["allowedOrigins"]`.

In [25]:
!az resource update --resource-group {resource_group} --name web --namespace Microsoft.Web --resource-type config --parent sites/{webapp_name} --set properties.cors.allowedOrigins={origins} --api-version 2015-06-01

{
  "id": "/subscriptions/0ca618d2-22a8-413a-96d0-0f1b531129c3/resourceGroups/cvbp_project_resources/providers/Microsoft.Web/sites/im-classif-resnet18-webapp",
  "identity": null,
  "kind": null,
  "location": "Central US",
  "managedBy": null,
  "name": "im-classif-resnet18-webapp",
  "plan": null,
  "properties": {
    "alwaysOn": true,
    "apiDefinition": null,
    "apiManagementConfig": null,
    "appCommandLine": "",
    "appSettings": null,
    "autoHealEnabled": false,
    "autoHealRules": null,
    "autoSwapSlotName": null,
    "azureMonitorLogCategories": null,
    "connectionStrings": null,
    "cors": {
      "allowedOrigins": [
        "YOUR-WEBAPP-ORIGIN"
      ],
      "supportCredentials": false
    },
    "customAppPoolIdentityAdminState": false,
    "customAppPoolIdentityTenantState": false,
    "defaultDocuments": [
      "Default.htm",
      "Default.html",
      "Default.asp",
      "index.htm",
      "index.html",
      "iisstart.htm",
      "default.aspx",
      

## 10. Clean up<a id="clean_up" />

If you would like to delete the webapp and the appservice plan, uncomment the two cells below and run the commands.

In [ ]:
#!az webapp delete --resource-group {resource_group} --name "im-classif-resnet18-webapp"

In [ ]:
#!az appservice plan delete --resource-group {resource_group} --name {appservice_plan_name} --yes